In [ ]:
jobContractsPath = "sample_data/JobContracts.txt"
jobOffersPath = "sample_data/JobOffers.txt"
jobPostingsPath = "sample_data/JobPostings.txt"

outputPath1 = "outSpark1/"
outputPath2 = "outSpark2/"

In [ ]:
jobPostingsRDD = sc.textFile( jobPostingsPath ) # JobID,Title,Country
jobOffersRDD = sc.textFile( jobOffersPath) # OfferID,JobID,Salary,Status,SSN
jobContractsRDD = sc.textFile( jobContractsPath ) #ContractID,OfferID,ContractDate,ContractType

In [ ]:
######################
# PART 1
######################

In [ ]:
# Filter accepted job offers and extract (JobID,(Salary, OfferId))
# OfferId is helpful for solving the second part.
# To avoid repeating the same join in the second part, we already retrieve the OfferId here.

def jobIDSalaryOfferID(line):
    fields = line.split(",")
    
    offerId = fields[0]
    jobID = fields[1]
    salary = float(fields[2])
    
    return (jobID, (salary,offerId))
    

acceptedOffers = jobOffersRDD\
    .filter(lambda line: line.find(",Accepted,")>=0)\
    .map(jobIDSalaryOfferID)

In [ ]:
# Extract (JobID, (Country, Title)) from job postings
# Title is helpful for solving the second part.
# To avoid repeating the same join in the second part, we already retrieve the title here.

def jobIDCountryTitle(line):
    fields = line.split(",")
    
    jobID = fields[0]
    title = fields[1]
    country = fields[2]
    
    return (jobID, (country,title))

jobIDCountry = jobPostingsRDD.map(jobIDCountryTitle)

In [ ]:
# Join accepted offers with job postings to get (JobID, ((Salary, OfferId), (Country, Title)))
offersWithCountry = acceptedOffers.join(jobIDCountry).cache()

In [ ]:
# Map to (Country, (Salary, 1))
countrySalaryCount = offersWithCountry\
    .map(lambda tuple: (tuple[1][1][0], (tuple[1][0][0], 1)))

In [ ]:
# Reduce by key to get (Country, (TotalSalary, Count))
countryTotalSalaryCount = countrySalaryCount\
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

In [ ]:
# Map to (Country, AverageSalary)
countryAverageSalary = countryTotalSalaryCount\
    .mapValues(lambda tuple: float(tuple[0]) / float(tuple[1]))

In [ ]:
# Select only the top N countries with the highest average salary
topCountries = countryAverageSalary\
    .top(3, lambda tuple: tuple[1])

In [ ]:
# Convert the top 3 countries to an RDD and save the result in the output folder
topCountriesRDD = sc.parallelize(topCountries)

topCountriesRDD.saveAsTextFile(outputPath1)

In [ ]:
######################
# PART 2
######################

In [ ]:
# Map offersWithCountry to
# (OfferID, (Country, Title))

def offIdTitleCountry(pair):
    offerId = pair[1][0][1]
    country = pair[1][1][0]
    title = pair[1][1][1]
    
    return (offerId, (title, country))
    

offerTitleCountry = offersWithCountry.map(offIdTitleCountry)

In [ ]:
# Map contracts to (OfferID, None)

def offIdNone(line):
    # ContractID,OfferID,ContractDate,ContractType
    fields = line.split(",")
    offerID = fields[1]
    
    return (offerID, None)
    

offerIdContract = jobContractsRDD.map(offIdNone)

In [ ]:
# Join offerTitleCountry with offerIdContract -> (offerId, ((Title, Country), null))
# map to ((title, country), +1)
# and apply reduceByKey to count the number of contracts for each title in each country
# ((title, country), numContracts)


titleCountryNumContracts = offerTitleCountry.join(offerIdContract)\
    .map(lambda pair: (pair[1][0], 1))\
    .reduceByKey(lambda v1, v2: v1+v2).cache()

In [ ]:
# Map to (Country, numContracts) and compute the maximum for each country
def CountryNumContracts(pair):
    country =  pair[0][1]
    numContracts = pair[1]
    
    return (country, numContracts)
    

countryMaxNumContracts = titleCountryNumContracts\
    .map(CountryNumContracts)\
    .reduceByKey(lambda v1, v2: max(v1,v2))

In [ ]:
countryMaxNumContracts.collect()

In [ ]:
# Map countryMaxNumContracts to 
# ( (Country, maxNumContracts), None )
countryMaxNull = countryMaxNumContracts.map(lambda pair: (pair, None))

In [ ]:
# Map Join titleCountryNumContracts to ((country, numContracts), title)

def CountryNumContractsTitle(pair):
    title = pair[0][0]
    country = pair[0][1]
    numContracts = pair[1]
    
    return ((country,numContracts), title)
    

countryNumContractsTitle = titleCountryNumContracts.map(CountryNumContractsTitle)

In [ ]:
# Join countryNumContractsTitle with countryMaxNull
# and map to the string Country,Title,NumberOfContracts
mostPopularTitlePerCountry = countryNumContractsTitle.join(countryMaxNull)\
    .map(lambda pair: pair[0][0]+ "," + pair[1][0] + "," + str(pair[0][1]))

In [ ]:
# Save the result to the output folder
mostPopularTitlePerCountry.saveAsTextFile(outputPath2)